<a href="https://colab.research.google.com/github/Kei-Sanada/Numerai/blob/master/236_Modeling_20201101_1820.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Round236にはこちらを提出

In [ ]:
# install dependencies
!pip install pandas sklearn numerapi

In [ ]:
!free -h 

              total        used        free      shared  buff/cache   available
Mem:            12G        544M         10G        964K        2.0G         11G
Swap:            0B          0B          0B


In [ ]:
import pandas as pd
import numerapi

In [ ]:
training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")
#training_data.head()

In [ ]:
tournament_data  = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
#tournament_data.head()

In [ ]:
#tournament_data['target_kazutsugi'].unique()

In [ ]:
# import dependencies
import sklearn.linear_model

In [ ]:
# find only the feature columns
feature_cols = [f for f in training_data.columns if "feature" in f]

# select those columns out of the training dataset
training_features = training_data[feature_cols]

In [ ]:
# create a model and fit the training data (~30 sec to run)
model = sklearn.linear_model.LinearRegression()
model.fit(training_features, training_data.target_kazutsugi)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# select the feature columns from the tournament data
live_features = tournament_data[feature_cols]

In [ ]:
# predict the target on the live features
predictions_sklearn_linear_model = model.predict(live_features)

正規化 https://forum.numer.ai/t/model-diagnostics-feature-exposure/899

In [ ]:
feature_names_dexterity = [f for f in training_data.columns if "feature_dexterity" in f]
feature_names_intelligence = [f for f in training_data.columns if "feature_intelligence" in f]


In [ ]:
feature_names_dexterity
feature_names_intelligence

['feature_intelligence1',
 'feature_intelligence2',
 'feature_intelligence3',
 'feature_intelligence4',
 'feature_intelligence5',
 'feature_intelligence6',
 'feature_intelligence7',
 'feature_intelligence8',
 'feature_intelligence9',
 'feature_intelligence10',
 'feature_intelligence11',
 'feature_intelligence12']

In [ ]:
 #training_data["target_kazutsugi"]

In [ ]:
 from xgboost import XGBRegressor

In [ ]:
# train a model to make predictions on tournament data
#feature_names_dexterity
model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity =1,
       alpha=0.1) #2020/11/01 alpha=0.1　重みに関するL1正則化項高次元の場合に用いるらしい。
model.fit(training_data[feature_names_dexterity], training_data["target_kazutsugi"])

# submit predictions to numer.ai
predictions_dexterity = model.predict(tournament_data[feature_names_dexterity])

In [ ]:
# train a model to make predictions on tournament data
#feature_names_intelligence
model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity =1,
       alpha=0.1) #2020/11/01 alpha=0.1　重みに関するL1正則化項高次元の場合に用いるらしい。
model.fit(training_data[feature_names_intelligence], training_data["target_kazutsugi"])

# submit predictions to numer.ai
predictions_intelligence = model.predict(tournament_data[feature_names_intelligence])

In [ ]:
predictions = (predictions_sklearn_linear_model + predictions_dexterity + predictions_intelligence) / 3

In [ ]:
#tournament_data["id"]

In [ ]:
# predictions must have an `id` column and a `prediction_kazutsugi` column
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions
predictions_df.head()

,id,prediction_kazutsugi
0,n0003aa52cab36c2,0.482960
1,n000920ed083903f,0.497361
2,n0038e640522c4a6,0.517235
3,n004ac94a87dc54b,0.499275
4,n0052fe97ea0c05f,0.495133


In [ ]:
import numpy as np
ranked_prediction = training_data["target_kazutsugi"].rank(pct=True, method="first")
correlation = np.corrcoef(training_data["target_kazutsugi"], ranked_prediction)[0, 1]

In [ ]:
ranked_prediction

0         0.601268
1         0.199365
2         0.000002
3         0.000004
4         0.601270
            ...   
501803    0.398732
501804    1.000000
501805    0.199363
501806    0.398734
501807    0.601266
Name: target_kazutsugi, Length: 501808, dtype: float64

In [ ]:
correlation

0.9797922144295415

In [ ]:
#training_data["target_kazutsugi"]

In [ ]:
# Get your API keys and model_id from https://numer.ai/submit
public_id = "CYATEL5QQBU6APNFLCV7HEE7PV6SC7V6"
secret_key = "Y22BTSUGU4JEFGQB3RZNEESSULKA3HQJPAW3KI6BIXH2AMNMCTC44IFWTOQIO2UW"
model_id = "3c77ba09-cfa2-4b18-b789-918340c84c82"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [ ]:
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=model_id)

2020-11-01 10:44:16,069 INFO numerapi.base_api: uploading predictions...


sklearn_linear_modelと　XGBRegressor

Performance
Validation Sharpe
0.3927
Validation Mean
0.0142
Feature Neutral Mean
0.0031
Risk
Validation SD
0.0362
Feature Exposure
0.4651
Max Drawdown
-0.1468
MMC
Corr + MMC Sharpe
0.1213
MMC Mean
-0.0072
Corr With Example Preds
0.7007

XGBRegressorのみ

Validation Sharpe
0.091
Validation Mean
0.0026
Feature Neutral Mean
0.0031
Risk
Validation SD
0.0287
Feature Exposure
0.4877
Max Drawdown
-0.1009
MMC
Corr + MMC Sharpe
-0.1684
MMC Mean
-0.0113
Corr With Example Preds
0.4248